In [1]:
import matplotlib.pyplot as plt
import os
from random import shuffle
from tqdm import tqdm
import tensorflow as tf
import cv2
import numpy as np 
from tensorflow.python.framework import graph_util
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import *
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [5]:
data1='D:\\fintest\\1coal'
data2='D:\\fintest\\2coal'
data3='D:\\fintest\\3coal'

In [6]:
def data_with_label():
    images=[]
    cnt = 0
    for i in tqdm(os.listdir(data1)):
        path=os.path.join(data1,i)
        img=cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img=cv2.resize(img,(64,64))
        images.append([np.array(img),np.array([1,0,0])])
    cnt = 0
    for i in tqdm(os.listdir(data2)):
        path=os.path.join(data2,i)
        img=cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img=cv2.resize(img,(64,64))
        images.append([np.array(img),np.array([0,1,0])])
    cnt = 0
    for i in tqdm(os.listdir(data3)):
        path=os.path.join(data3,i)
        img=cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img=cv2.resize(img,(64,64))
        images.append([np.array(img),np.array([0,0,1])])
        
    shuffle(images)
    return images

In [7]:
data = data_with_label()

100%|████████████████████████████████████████████████████████████████████████████████| 455/455 [00:50<00:00,  9.08it/s]


In [8]:
import random
random.shuffle(data)
train_data, test_data = train_test_split(data, test_size=0.2)

In [9]:
tr_img_data = np.array([i[0] for i in train_data]).reshape(-1,64,64,1)
tr_img_data = tr_img_data/255
tr_lbl_data = np.array([i[1] for i in train_data])
tst_img_data = np.array([i[0] for i in test_data]).reshape(-1,64,64,1)
tst_img_data = tst_img_data/255
tst_lbl_data = np.array([i[1] for i in test_data])

In [10]:
def create_model():
  model=Sequential()
  model.add(InputLayer(input_shape=[64,64,1]))

  model.add(Conv2D(filters=32,kernel_size=5,strides=1,padding='same',activation='relu'))
  model.add(MaxPool2D(pool_size=5,padding='same'))
          
  model.add(Conv2D(filters=32,kernel_size=5,strides=2,padding='same',activation='relu'))
  model.add(MaxPool2D(pool_size=5,padding='same'))
          
  model.add(Conv2D(filters=64,kernel_size=3,strides=2,padding='same',activation='relu'))
  model.add(MaxPool2D(pool_size=8,padding='same'))

  model.add(Conv2D(filters=128,kernel_size=3,strides=1,padding='same',activation='relu'))
  model.add(MaxPool2D(pool_size=5,padding='same'))
          
#model.add(Conv2D(filters=256,kernel_size=3,strides=1,padding='same',activation='relu'))
#model.add(MaxPool2D(pool_size=3,padding='same'))
          

  model.add(Flatten())


  model.add(Dense(2048,activation='relu'))
  model.add(Dropout(0.50))
  model.add(Dense(1024,activation='relu'))
  model.add(Dropout(0.50))
  model.add(Dense(512,activation='relu'))
  model.add(Dropout(0.50))
  model.add(Dense(3,activation='softmax'))
  optimizer=Adam(lr=1e-3)
          
  model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])
  return model
#history = model.fit(x=tr_img_data,y=tr_lbl_data,epochs=50,batch_size=32,validation_data=(tst_img_data, tst_lbl_data),shuffle=True)
model = create_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 32)        832       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 32)          25632     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 2, 2, 32)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 1, 1, 64)          18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 1, 1, 64)          0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 1, 1, 128)         73856     
__________

In [13]:
checkpoint_path = "cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(tr_img_data, 
          tr_lbl_data,  
          epochs=50,
          validation_data=(tst_img_data,tst_lbl_data),
          callbacks=[cp_callback])  # Pass callback to training


Train on 915 samples, validate on 229 samples
Epoch 1/50
915/915 [==============================] - ETA: 17s - loss: 1.1016 - acc: 0.28 - ETA: 20s - loss: 1.0947 - acc: 0.34 - ETA: 20s - loss: 1.0933 - acc: 0.36 - ETA: 18s - loss: 1.0903 - acc: 0.35 - ETA: 18s - loss: 1.0922 - acc: 0.36 - ETA: 17s - loss: 1.0897 - acc: 0.38 - ETA: 17s - loss: 1.0873 - acc: 0.38 - ETA: 16s - loss: 1.0844 - acc: 0.39 - ETA: 15s - loss: 1.0844 - acc: 0.38 - ETA: 14s - loss: 1.0842 - acc: 0.37 - ETA: 13s - loss: 1.0841 - acc: 0.38 - ETA: 12s - loss: 1.0791 - acc: 0.39 - ETA: 12s - loss: 1.0815 - acc: 0.38 - ETA: 11s - loss: 1.0800 - acc: 0.38 - ETA: 10s - loss: 1.0775 - acc: 0.39 - ETA: 9s - loss: 1.0781 - acc: 0.3984 - ETA: 8s - loss: 1.0790 - acc: 0.402 - ETA: 8s - loss: 1.0801 - acc: 0.395 - ETA: 7s - loss: 1.0855 - acc: 0.394 - ETA: 6s - loss: 1.0886 - acc: 0.392 - ETA: 5s - loss: 1.0889 - acc: 0.389 - ETA: 5s - loss: 1.0891 - acc: 0.387 - ETA: 4s - loss: 1.0891 - acc: 0.384 - ETA: 3s - loss: 1.0871 - 

915/915 [==============================] - ETA: 23s - loss: 0.6247 - acc: 0.71 - ETA: 26s - loss: 0.7165 - acc: 0.70 - ETA: 25s - loss: 0.6893 - acc: 0.69 - ETA: 24s - loss: 0.6708 - acc: 0.70 - ETA: 21s - loss: 0.6764 - acc: 0.69 - ETA: 20s - loss: 0.6754 - acc: 0.67 - ETA: 19s - loss: 0.6847 - acc: 0.68 - ETA: 18s - loss: 0.6672 - acc: 0.68 - ETA: 18s - loss: 0.6667 - acc: 0.68 - ETA: 17s - loss: 0.6519 - acc: 0.69 - ETA: 16s - loss: 0.6387 - acc: 0.69 - ETA: 15s - loss: 0.6400 - acc: 0.69 - ETA: 13s - loss: 0.6393 - acc: 0.70 - ETA: 12s - loss: 0.6404 - acc: 0.70 - ETA: 11s - loss: 0.6269 - acc: 0.71 - ETA: 10s - loss: 0.6190 - acc: 0.71 - ETA: 10s - loss: 0.6268 - acc: 0.71 - ETA: 9s - loss: 0.6280 - acc: 0.7205 - ETA: 8s - loss: 0.6338 - acc: 0.717 - ETA: 7s - loss: 0.6255 - acc: 0.721 - ETA: 6s - loss: 0.6191 - acc: 0.729 - ETA: 5s - loss: 0.6181 - acc: 0.733 - ETA: 4s - loss: 0.6195 - acc: 0.732 - ETA: 3s - loss: 0.6252 - acc: 0.730 - ETA: 3s - loss: 0.6268 - acc: 0.727 - ETA: 2

915/915 [==============================] - ETA: 21s - loss: 0.5522 - acc: 0.71 - ETA: 20s - loss: 0.5023 - acc: 0.78 - ETA: 19s - loss: 0.4378 - acc: 0.80 - ETA: 18s - loss: 0.5702 - acc: 0.74 - ETA: 18s - loss: 0.5560 - acc: 0.73 - ETA: 17s - loss: 0.5227 - acc: 0.75 - ETA: 16s - loss: 0.5407 - acc: 0.75 - ETA: 15s - loss: 0.5463 - acc: 0.74 - ETA: 15s - loss: 0.5200 - acc: 0.76 - ETA: 13s - loss: 0.4958 - acc: 0.78 - ETA: 13s - loss: 0.4884 - acc: 0.78 - ETA: 12s - loss: 0.4877 - acc: 0.79 - ETA: 11s - loss: 0.4724 - acc: 0.80 - ETA: 11s - loss: 0.4573 - acc: 0.81 - ETA: 10s - loss: 0.4441 - acc: 0.82 - ETA: 9s - loss: 0.4345 - acc: 0.8281 - ETA: 8s - loss: 0.4207 - acc: 0.832 - ETA: 8s - loss: 0.4054 - acc: 0.838 - ETA: 7s - loss: 0.4131 - acc: 0.833 - ETA: 6s - loss: 0.4100 - acc: 0.835 - ETA: 5s - loss: 0.4189 - acc: 0.831 - ETA: 4s - loss: 0.4160 - acc: 0.832 - ETA: 4s - loss: 0.4092 - acc: 0.834 - ETA: 3s - loss: 0.4055 - acc: 0.834 - ETA: 2s - loss: 0.4104 - acc: 0.831 - ETA: 1

915/915 [==============================] - ETA: 21s - loss: 0.1845 - acc: 0.90 - ETA: 20s - loss: 0.1288 - acc: 0.93 - ETA: 20s - loss: 0.2348 - acc: 0.90 - ETA: 18s - loss: 0.2581 - acc: 0.89 - ETA: 17s - loss: 0.2192 - acc: 0.91 - ETA: 17s - loss: 0.1936 - acc: 0.92 - ETA: 16s - loss: 0.1754 - acc: 0.93 - ETA: 16s - loss: 0.1837 - acc: 0.92 - ETA: 15s - loss: 0.1723 - acc: 0.93 - ETA: 14s - loss: 0.1649 - acc: 0.93 - ETA: 13s - loss: 0.1673 - acc: 0.94 - ETA: 12s - loss: 0.1645 - acc: 0.94 - ETA: 11s - loss: 0.1553 - acc: 0.94 - ETA: 11s - loss: 0.1632 - acc: 0.94 - ETA: 10s - loss: 0.1559 - acc: 0.94 - ETA: 9s - loss: 0.1642 - acc: 0.9395 - ETA: 9s - loss: 0.1685 - acc: 0.939 - ETA: 8s - loss: 0.1654 - acc: 0.942 - ETA: 7s - loss: 0.1684 - acc: 0.942 - ETA: 7s - loss: 0.1761 - acc: 0.937 - ETA: 6s - loss: 0.1813 - acc: 0.933 - ETA: 5s - loss: 0.1845 - acc: 0.931 - ETA: 4s - loss: 0.1929 - acc: 0.928 - ETA: 3s - loss: 0.1899 - acc: 0.929 - ETA: 3s - loss: 0.1875 - acc: 0.931 - ETA: 2

915/915 [==============================] - ETA: 24s - loss: 0.0462 - acc: 0.96 - ETA: 24s - loss: 0.1382 - acc: 0.96 - ETA: 25s - loss: 0.1731 - acc: 0.94 - ETA: 24s - loss: 0.1330 - acc: 0.96 - ETA: 22s - loss: 0.1179 - acc: 0.96 - ETA: 22s - loss: 0.1147 - acc: 0.96 - ETA: 20s - loss: 0.1191 - acc: 0.95 - ETA: 19s - loss: 0.1158 - acc: 0.95 - ETA: 18s - loss: 0.1090 - acc: 0.96 - ETA: 17s - loss: 0.1263 - acc: 0.95 - ETA: 16s - loss: 0.1284 - acc: 0.95 - ETA: 14s - loss: 0.1380 - acc: 0.95 - ETA: 13s - loss: 0.1360 - acc: 0.95 - ETA: 12s - loss: 0.1433 - acc: 0.94 - ETA: 11s - loss: 0.1526 - acc: 0.94 - ETA: 10s - loss: 0.1474 - acc: 0.94 - ETA: 10s - loss: 0.1413 - acc: 0.94 - ETA: 9s - loss: 0.1401 - acc: 0.9497 - ETA: 8s - loss: 0.1401 - acc: 0.949 - ETA: 7s - loss: 0.1372 - acc: 0.950 - ETA: 6s - loss: 0.1356 - acc: 0.950 - ETA: 5s - loss: 0.1380 - acc: 0.950 - ETA: 4s - loss: 0.1333 - acc: 0.952 - ETA: 4s - loss: 0.1282 - acc: 0.954 - ETA: 3s - loss: 0.1236 - acc: 0.956 - ETA: 2

915/915 [==============================] - ETA: 21s - loss: 0.0054 - acc: 1.00 - ETA: 22s - loss: 0.0075 - acc: 1.00 - ETA: 21s - loss: 0.0078 - acc: 1.00 - ETA: 21s - loss: 0.0075 - acc: 1.00 - ETA: 19s - loss: 0.0066 - acc: 1.00 - ETA: 19s - loss: 0.0060 - acc: 1.00 - ETA: 18s - loss: 0.0149 - acc: 0.99 - ETA: 18s - loss: 0.0227 - acc: 0.98 - ETA: 17s - loss: 0.0266 - acc: 0.98 - ETA: 16s - loss: 0.0245 - acc: 0.98 - ETA: 15s - loss: 0.0263 - acc: 0.98 - ETA: 14s - loss: 0.0250 - acc: 0.98 - ETA: 13s - loss: 0.0284 - acc: 0.99 - ETA: 13s - loss: 0.0308 - acc: 0.98 - ETA: 12s - loss: 0.0289 - acc: 0.98 - ETA: 11s - loss: 0.0278 - acc: 0.99 - ETA: 10s - loss: 0.0700 - acc: 0.97 - ETA: 9s - loss: 0.0676 - acc: 0.9740 - ETA: 8s - loss: 0.0928 - acc: 0.965 - ETA: 7s - loss: 0.1148 - acc: 0.957 - ETA: 6s - loss: 0.1136 - acc: 0.956 - ETA: 5s - loss: 0.1212 - acc: 0.954 - ETA: 4s - loss: 0.1737 - acc: 0.941 - ETA: 4s - loss: 0.1689 - acc: 0.944 - ETA: 3s - loss: 0.1655 - acc: 0.945 - ETA: 2

915/915 [==============================] - ETA: 23s - loss: 1.4214 - acc: 0.68 - ETA: 21s - loss: 0.8533 - acc: 0.76 - ETA: 19s - loss: 0.5880 - acc: 0.84 - ETA: 18s - loss: 0.4744 - acc: 0.86 - ETA: 18s - loss: 0.6149 - acc: 0.83 - ETA: 17s - loss: 0.5899 - acc: 0.83 - ETA: 17s - loss: 0.5239 - acc: 0.85 - ETA: 16s - loss: 0.4876 - acc: 0.86 - ETA: 16s - loss: 0.4626 - acc: 0.86 - ETA: 15s - loss: 0.4341 - acc: 0.87 - ETA: 14s - loss: 0.4181 - acc: 0.87 - ETA: 13s - loss: 0.3917 - acc: 0.88 - ETA: 12s - loss: 0.3763 - acc: 0.88 - ETA: 11s - loss: 0.3619 - acc: 0.88 - ETA: 11s - loss: 0.3499 - acc: 0.88 - ETA: 10s - loss: 0.3397 - acc: 0.88 - ETA: 9s - loss: 0.3359 - acc: 0.8897 - ETA: 8s - loss: 0.3227 - acc: 0.895 - ETA: 7s - loss: 0.3134 - acc: 0.899 - ETA: 6s - loss: 0.3024 - acc: 0.904 - ETA: 6s - loss: 0.2990 - acc: 0.903 - ETA: 5s - loss: 0.2903 - acc: 0.906 - ETA: 4s - loss: 0.2815 - acc: 0.910 - ETA: 3s - loss: 0.2739 - acc: 0.912 - ETA: 2s - loss: 0.2663 - acc: 0.915 - ETA: 2

915/915 [==============================] - ETA: 21s - loss: 0.0019 - acc: 1.00 - ETA: 18s - loss: 0.0012 - acc: 1.00 - ETA: 19s - loss: 0.0011 - acc: 1.00 - ETA: 18s - loss: 0.0011 - acc: 1.00 - ETA: 18s - loss: 0.0014 - acc: 1.00 - ETA: 17s - loss: 0.0037 - acc: 1.00 - ETA: 17s - loss: 0.0044 - acc: 1.00 - ETA: 16s - loss: 0.0042 - acc: 1.00 - ETA: 15s - loss: 0.0038 - acc: 1.00 - ETA: 14s - loss: 0.0107 - acc: 0.99 - ETA: 13s - loss: 0.0099 - acc: 0.99 - ETA: 12s - loss: 0.0091 - acc: 0.99 - ETA: 12s - loss: 0.0095 - acc: 0.99 - ETA: 11s - loss: 0.0148 - acc: 0.99 - ETA: 11s - loss: 0.0153 - acc: 0.99 - ETA: 10s - loss: 0.0145 - acc: 0.99 - ETA: 9s - loss: 0.0137 - acc: 0.9963 - ETA: 8s - loss: 0.0130 - acc: 0.996 - ETA: 8s - loss: 0.0124 - acc: 0.996 - ETA: 7s - loss: 0.0163 - acc: 0.995 - ETA: 6s - loss: 0.0156 - acc: 0.995 - ETA: 5s - loss: 0.0156 - acc: 0.995 - ETA: 4s - loss: 0.0150 - acc: 0.995 - ETA: 4s - loss: 0.0144 - acc: 0.996 - ETA: 3s - loss: 0.0153 - acc: 0.995 - ETA: 2

915/915 [==============================] - ETA: 18s - loss: 0.3504 - acc: 0.93 - ETA: 20s - loss: 0.1840 - acc: 0.96 - ETA: 18s - loss: 0.1251 - acc: 0.97 - ETA: 17s - loss: 0.0952 - acc: 0.98 - ETA: 16s - loss: 0.0986 - acc: 0.98 - ETA: 16s - loss: 0.0823 - acc: 0.98 - ETA: 15s - loss: 0.0713 - acc: 0.98 - ETA: 15s - loss: 0.0798 - acc: 0.98 - ETA: 14s - loss: 0.0715 - acc: 0.98 - ETA: 13s - loss: 0.0646 - acc: 0.98 - ETA: 13s - loss: 0.0590 - acc: 0.98 - ETA: 12s - loss: 0.0543 - acc: 0.98 - ETA: 11s - loss: 0.0536 - acc: 0.98 - ETA: 10s - loss: 0.0505 - acc: 0.98 - ETA: 10s - loss: 0.0555 - acc: 0.98 - ETA: 9s - loss: 0.0535 - acc: 0.9883 - ETA: 8s - loss: 0.0505 - acc: 0.989 - ETA: 7s - loss: 0.0500 - acc: 0.989 - ETA: 7s - loss: 0.0508 - acc: 0.988 - ETA: 6s - loss: 0.0612 - acc: 0.985 - ETA: 5s - loss: 0.0601 - acc: 0.985 - ETA: 4s - loss: 0.0580 - acc: 0.985 - ETA: 4s - loss: 0.0570 - acc: 0.986 - ETA: 3s - loss: 0.0589 - acc: 0.984 - ETA: 2s - loss: 0.0617 - acc: 0.983 - ETA: 1

In [14]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(tst_img_data,  tst_lbl_data, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

Untrained model, accuracy: 39.30%


In [15]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss,acc = model.evaluate(tst_img_data,  tst_lbl_data, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

Restored model, accuracy: 82.53%


In [46]:
img = cv2.imread("D:\\fintest\\2coal\\pic2crop.jpg",cv2.IMREAD_GRAYSCALE)
img=cv2.resize(img,(64,64))
img=np.array(img)
img =np.array(img).reshape(-1,64,64,1)
img = img/255
model_out=model.predict([img])
print(model_out)
if np.argmax(model_out)==0:
    str_label='class1'
        
elif np.argmax(model_out)==1:
    str_label='class2'
else:
    str_label='class3'
print(str_label)


[[0.00120043 0.9956606  0.00313892]]
class2
